In [ ]:
# Для каждой из шести задач прогнозирования сформируйте выборки. 
#   Откликом будет yT+iy_{T+i}yT+i​ при всевозможных значениях TTT, а признаки можно использовать следующие:

# идентификатор географической зоны — категориальный
# год, месяц, день месяца, день недели, час 
# синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
# сами значения прогнозов ARIMA
# количество поездок из рассматриваемого района в моменты времени Yt
# количество поездок из рассматриваемого района в моменты времени Yt-24
# суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from datetime import timedelta

In [ ]:
# Чтение исходных данных

In [2]:
raw_data = pd.read_csv('data/df_result2.csv')
raw_data.set_index('date', inplace=True)
raw_data.sort_index(inplace=True)

raw_data.head()

,start_region,trips_count
date,,
2015-01-01 00:00:00,2500,0
2015-01-01 00:00:00,1663,0
2015-01-01 00:00:00,1664,0
2015-01-01 00:00:00,1665,0
2015-01-01 00:00:00,1666,0


In [4]:
# данные хранились в двух столбцах, развернем в сводную таблицу по ячейкам
raw_data2 = pd.pivot_table(raw_data, index='date', columns='start_region', values='trips_count', aggfunc=sum)
raw_data2.index = pd.to_datetime(raw_data2.index)
raw_data2.head()

start_region,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_may = raw_data2.loc['2016-05-01 00:00:00':'2016-05-31 23:00:00', :]
(data_may.mean()>5).sum()

102

In [9]:
# список интересующих ячеек
r_list = data_may.loc[:, data_may.mean()>5].columns

In [24]:
filtered_data = raw_data2.loc['2016-01-01 00:00:00':, r_list]
filtered_data.head(2)

start_region,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,80,144,50,77,319,402,531,617,846,267,...,12,0,2,44,5,41,4,70,7,66
2016-01-01 01:00:00,91,211,49,134,404,420,370,453,594,224,...,29,0,5,2,2,4,0,47,1,29


In [25]:
data = filtered_data.copy()

In [ ]:
# генерация временных признаков

In [26]:
# месяц, день месяца, день недели, чаc
data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
data['day of week'] = data.index.dayofweek
data['hour'] = data.index.hour

# годовые, месячные и суточные признаки Фурье
K = 15
year_period = 8766 #365.2424*24 - средняя продолжительность года в часах
week_period = 168
day_period = 24
for i in range(1, K+1):
    data['y_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
    data['y_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/year_period)
for i in range(1, K+1):    
    data['w_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
    data['w_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/week_period)
for i in range(1, K+1):    
    data['d_с_' + str(i)] = np.cos(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)
    data['d_s_' + str(i)] = np.sin(np.arange(1, data.shape[0]+1)*2*np.pi*i/day_period)  

In [14]:
# **Создание регрессионных моделей для каждого региона**

In [27]:
%%time

import warnings
warnings.filterwarnings('ignore')

linear_models_dict = {}

X_val_dict = {}
y_val_dict = {}
y_val_pred_dict = {}

X_test_dict = {}
y_test_dict = {}
y_test_pred_dict = {}

for id_name in filtered_data.columns:
    print(id_name,)
    drop_list = list(filtered_data.columns)
    drop_list.remove(id_name)
    named_data = data.copy().drop(drop_list, axis = 1) #Избавляемся от лишних id на итерации
    
    #Добавляем shift признаки
    named_data['shift_1'] = named_data[id_name].shift(1)
    named_data['shift_2'] = named_data[id_name].shift(2)
    named_data['shift_3'] = named_data[id_name].shift(3)
    named_data['shift_4'] = named_data[id_name].shift(4)
    named_data['shift_5'] = named_data[id_name].shift(5)
    named_data['shift_6'] = named_data[id_name].shift(6)
    named_data['shift_24'] = named_data[id_name].shift(24)
    named_data['shift_48'] = named_data[id_name].shift(48)
    
    named_data = named_data.iloc[48:]
    
    # Разбиение данных
    predict_may_start = '2016-04-30 23:00'
    predict_may_stop = '2016-05-31 23:00'
    precit_june_stop = '2016-06-30 23:00' 
    
    X_train = named_data.loc[:predict_may_start].drop(id_name, axis = 1)
    y_train = named_data.loc[:predict_may_start][id_name]
    
    X_val = named_data.loc[predict_may_start:predict_may_stop].drop(id_name, axis = 1)
    y_val = named_data.loc[predict_may_start:predict_may_stop][id_name]
    
    X_test = named_data.loc[predict_may_stop:].drop(id_name, axis = 1)
    y_test = named_data.loc[predict_may_stop:][id_name]
    
    #Строим Ridge модель с подбором гиперпараметра
    model = linear_model.Ridge()
    param_grid = [{'alpha' : np.logspace(2, 3, 50)}]
    clf = GridSearchCV(model, param_grid, cv = 5)
    best_model = clf.fit(X_train, y_train)
    
    # Сохраним результаты разбиения данных, моделей и их прогнозы по словарям. Ключлм будет являться id региона
    linear_models_dict[id_name] = best_model
    X_val_dict[id_name] = X_val
    y_val_dict[id_name] = y_val
    y_val_pred_dict[id_name] = best_model.predict(X_val)
    X_test_dict[id_name] = X_test
    y_test_dict[id_name] = y_test
    y_test_pred_dict[id_name] = best_model.predict(X_test)
       

1075
1076
1077
1125
1126
1127
1128
1129
1130
1131
1132
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1221
1222
1223
1224
1225
1227
1228
1229
1230
1231
1232
1233
1234
1235
1272
1273
1274
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1326
1327
1331
1332
1333
1334
1335
1336
1337
1338
1339
1376
1377
1378
1380
1382
1383
1384
1385
1386
1387
1388
1389
1390
1426
1431
1434
1435
1436
1437
1438
1439
1441
1442
1480
1482
1483
1530
1532
1533
1580
1630
1684
1733
1734
1783
2068
2069
2118
2119
2168
Wall time: 2min 34s


In [28]:
# Использовать SARIMA признаки не получилось, так как для этого нужно прогонять SARIMA модель через весь описываемый датасетом промежуток времени.

In [29]:
# Анализ предсказаний за май
filtered_data.columns

Int64Index([1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131,
            ...
            1630, 1684, 1733, 1734, 1783, 2068, 2069, 2118, 2119, 2168],
           dtype='int64', name='start_region', length=102)

In [30]:
# Временной диапазон
may_time_range = pd.date_range('2016-04-30 23:00', '2016-05-31 17:00', freq = 'H')

In [20]:
%%time
Q_may_f = 0
for key in filtered_data.columns:
    print(key,)
    for date in may_time_range:
        for step in range(1, 7):
            # создадим вспомогательный df для прогноза, чтобы использовать индексы от оригинального df
            forecast = pd.DataFrame(y_val_pred_dict.get(key), index = y_val_dict.get(key).index, columns = ['val'])
            real_values = y_val_dict.get(key)
            Q_may_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_may = float(Q_may_f/(102*739*6))
print() 
print('Q_may: ', Q_may)

1075
1076
1077
1125
1126
1127
1128
1129
1130
1131
1132
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1221
1222
1223
1224
1225
1227
1228
1229
1230
1231
1232
1233
1234
1235
1272
1273
1274
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1326
1327
1331
1332
1333
1334
1335
1336
1337
1338
1339
1376
1377
1378
1380
1382
1383
1384
1385
1386
1387
1388
1389
1390
1426
1431
1434
1435
1436
1437
1438
1439
1441
1442
1480
1482
1483
1530
1532
1533
1580
1630
1684
1733
1734
1783
2068
2069
2118
2119
2168

Q_may:  17.576686954903536
Wall time: 2min 7s


In [ ]:
# анализ предсказаний за июнь
june_time_range = pd.date_range('2016-05-31 23:00', '2016-06-30 17:00', freq = 'H')

In [33]:
%%time
Q_june_f = 0
for key in filtered_data.columns:
    print(key,end = ' ')
    for date in june_time_range:
        for step in range(1, 7):
            forecast = pd.DataFrame(y_test_pred_dict.get(key), index = y_test_dict.get(key).index, columns = ['val'])
            real_values = y_test_dict.get(key)
            Q_june_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_june = float(Q_june_f/(102*739*6))
print()
print('Q_june: ', Q_june)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
Q_june:  16.867145286940882
Wall time: 2min 3s


In [35]:
%%time
Q_june_f = 0
title_list = []
prediction_list = []
for key in filtered_data.columns:
    print(key,end =' ')
    for date in june_time_range:
        for step in range(1, 7):
            forecast = pd.DataFrame(y_test_pred_dict.get(key), index = y_test_dict.get(key).index, columns = ['val'])
            real_values = y_test_dict.get(key)           
                
            prediction_list.append(float(forecast.loc[date + timedelta(hours = step)]))
            title_list.append(str(key) + '_' + str(date)[:10]+ '_' + str(date.hour) + '_' + str(step))  
            
            Q_june_f+= np.abs(forecast.loc[date + timedelta(hours = step)] - real_values.loc[date + timedelta(hours = step)])
Q_june = float(Q_june_f/(102*739*6))
print()
print('Q_june: ', Q_june)

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168 
Q_june:  16.867145286940882
Wall time: 2min 31s


In [36]:
kaggle_week5_df = pd.DataFrame(prediction_list, index = title_list, columns = ['y'])
kaggle_week5_df.index.name = 'id'
print(kaggle_week5_df.shape)
kaggle_week5_df.head()

(437580, 1)


,y
id,
1075_2016-05-31_23_1,22.635483
1075_2016-05-31_23_2,14.230841
1075_2016-05-31_23_3,5.235014
1075_2016-05-31_23_4,3.513301
1075_2016-05-31_23_5,6.073159


In [37]:
kaggle_week5_df.to_csv('kdw5.csv')

In [39]:
# https://inclass.kaggle.com/c/yellowtaxi/leaderboard